<a href="https://colab.research.google.com/github/syoung7388/Traffic_sign_recognition/blob/main/STN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spatial transformer networks**
#####  이미지의 관심 영역을 잘라내거나, 크기를 조정하거나, 방향을 수정하는 것이다. 이러한 회전, 크기 조정 등의 일반적인 아핀(Affine) 변환된 입력에 대해 결과의 변동이 크기 때문에, STN은 이를 극복하는데 매우 유용한 메커니즘이다. STN이 가진 장점중 하나는 아주 작은 수정만으로 기존에 사용하던 CNN에 간단하게 연결 시킬 수 있다는 것이다.

In [ ]:
# import here
from google.colab import drive
import os 
import PIL
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader 
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torch.utils.data.sampler as sampler
from torch import nn, optim
import torch.nn.functional as F



drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Stn(nn.Module):
  def __init__(self):
    super(Stn, self).__init__()
    self.loc_net = nn.Sequential(
        nn.Conv2d(1, 50, 7), 
        nn.MaxPool2d(2, 2), 
        nn.ELU(), 
        nn.Conv2d(50, 100, 5), 
        nn.MaxPool2d(2, 2), 
        nn.ELU()
    )

    self.fc_loc = nn.Sequential(
        nn.Linear(100*4*4, 100), 
        nn.ELU(), 
        nn.Linear(100, 3*2)
    )

    self.fc_loc[2].weight.data.zero_()
    self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype = torch.float))


    def forward(self, x):
      xs = self.loc_net(x)
      xs = xs.view(-1, 100*4*4)
      theta = self.fc_loc(xs)
      theta = theta.view(-1, 2, 3)

      grid = F.affine_grid(theta, x.size())
      x = F.grid_sample(x, grid)
      return x

class TrafficSignNet(nn.Module):
  def __init__(self):
    super(TrafficSignNet, self).__init__()

    self.stn = Stn()
    
    self.conv1 = nn.Conv2d(1, 100, 5)
    self.conv1_bn = nn.BatchNorm2d(100) # 정규화를 입력에 적용하여 평균 및 단위 분산이 0이 되고 네트워크 정확도를 높입니다.
    self.pool = nn.MaxPool2d(2, 2)
    
    self.conv2 = nn.Conv2d(100, 150, 3)
    self.conv2_bn = nn.BatchNorm2d(150)
    
    self.conv3 = nn.Conv2d(150, 250, 1)
    self.conv3_bn = nn.BatchNorm2d(250)


    self.fc1 = nn.Linear(250*3*3, 350)
    self.fc1_bn = nn.BatchNorm1d(350)
    
    self.fc2 = nn.Linear(350, 43)


    self.dropout = nn.Dropout(p = 0.5)



  def forward(self, x):
      x = self.pool(F.elu(self.conv1(x)))
      x = self.dropout(self.conv1_bn(x))
      x = self.pool(F.elu(self.conv2(x)))
      x = self.dropout(self.conv2_bn(x))
      x = self.pool(F.elu(self.conv3(x)))
      x = self.dropout(self.conv3_bn(x))
      x = x.view(-1, 250 * 3 * 3)
      x = F.elu(self.fc1(x))
      x = self.dropout(self.fc1_bn(x))
      x = self.fc2(x)
      return x
    

In [ ]:
def loss_batch(model, loss_func, x, y, opt = None):
    loss = loss_func(model(x), y)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
    return loss.item(), len(x)
def valid_batch(model, loss_func, x, y):
    output = model(x)
    loss = loss_func(output, y)
    pred = torch.argmax(output, dim = 1)
    correct = pred == y.view(*pred.shape)
    return loss.item(), torch.sum(correct).item(), len(x)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        
        #Train model
        model.train()
        losses, nums = zip(*[loss_batch(model, loss_func, x, y, opt) for x, y in train_dl])
        train_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        #Validation model
        model.eval()
        with torch.no_grad():
            losses, corrects, nums = zip(*[valid_batch(model, loss_func, x, y) for x, y in valid_dl])
            valid_loss = np.sum(np.multiply(losses, nums) / np.sum(nums))
            valid_accuracy = np.sum(corrects) / np.sum(nums)*100
            print(f"[Epoch {epoch+1}/{epochs}] "
                  f"Train loss: {train_loss:.6f}\t"
                  f"Validation loss: {valid_loss:.6f}\t",
                  f"Validation accruacy: {valid_accuracy:.3f}%")            
def evaluate(model, loss_func, dl):
    model.eval()
    with torch.no_grad():
        losses, corrects, nums = zip(*[valid_batch(model, loss_func, x, y) for x, y in dl])
        test_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        test_accuracy = np.sum(corrects) / np.sum(nums) * 100
        
    print(f"Test loss: {test_loss:.6f}\t"
          f"Test accruacy: {test_accuracy:.3f}%")
    

In [ ]:

#Custom DataSet & DataLoader 
class PickledDataset(Dataset):
    def __init__(self, file_path, transform = None):
        with open(file_path, mode = 'rb') as f:
            data = pickle.load(f)
            self.features = data['features']
            self.labels = data['labels']
            self.count = len(self.labels)
            self.transform = transform
    def __getitem__(self, index):
        feature = self.features[index]
        if self.transform is not None:
            feature = self.transform(feature)
        return (feature, self.labels[index])
    
    def __len__(self):
        return self.count
    
class WrappendDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func
    
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))






#Load Data


training_file = '/content/drive/MyDrive/Traffic/data/train_gray.p'
validation_file = '/content/drive/MyDrive/Traffic/data/valid_gray.p'
testing_file = '/content/drive/MyDrive/Traffic/data/test_gray.p'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def to_device(x, y):
    return x.to(device), y.to(device, dtype = torch.int64)

data_transforms = transforms.Compose([
    transforms.ToTensor()                                      
])
train_dataset = PickledDataset(training_file, transform =data_transforms )
valid_dataset = PickledDataset(validation_file, transform = data_transforms)
test_dataset = PickledDataset(testing_file, transform = data_transforms)



n_epochs = 20
n_train = len(train_dataset.features)
n_classes = len(set(train_dataset.labels))

train_loader = WrappendDataLoader(DataLoader(train_dataset, batch_size = 64, shuffle = True), to_device)
valid_loader = WrappendDataLoader(DataLoader(valid_dataset, batch_size = 64, shuffle = False),to_device)
test_loader = WrappendDataLoader(DataLoader(test_dataset, batch_size = 64, shuffle = False),to_device)



In [ ]:

#Train model 

model = TrafficSignNet().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()
fit(n_epochs, model, criterion, optimizer, train_loader, valid_loader) 

[Epoch 1/20] Train loss: 2.276792	Validation loss: 1.320466	 Validation accruacy: 64.150%
[Epoch 2/20] Train loss: 1.047861	Validation loss: 0.738623	 Validation accruacy: 80.930%
[Epoch 3/20] Train loss: 0.657835	Validation loss: 0.462388	 Validation accruacy: 87.143%
[Epoch 4/20] Train loss: 0.450904	Validation loss: 0.317950	 Validation accruacy: 91.791%
[Epoch 5/20] Train loss: 0.337887	Validation loss: 0.240050	 Validation accruacy: 93.946%
[Epoch 6/20] Train loss: 0.261630	Validation loss: 0.199844	 Validation accruacy: 95.306%
[Epoch 7/20] Train loss: 0.215207	Validation loss: 0.173271	 Validation accruacy: 95.669%
[Epoch 8/20] Train loss: 0.174723	Validation loss: 0.142439	 Validation accruacy: 96.168%
[Epoch 9/20] Train loss: 0.152137	Validation loss: 0.132627	 Validation accruacy: 96.349%
[Epoch 10/20] Train loss: 0.130859	Validation loss: 0.125999	 Validation accruacy: 96.485%
[Epoch 11/20] Train loss: 0.116451	Validation loss: 0.105677	 Validation accruacy: 96.916%
[Epoch 1

In [ ]:
# Test model
evaluate(model, criterion, test_loader)

Test loss: 0.089112	Test accruacy: 97.443%
